<a href="https://colab.research.google.com/github/ihatethinkingname/dl_practice/blob/main/sasrec_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


import copy
# class SASRec(nn.Module):
#   def __init__(self, item_num) -> None:
#     super().__init__()

class EnD(nn.Module):
  def __init__(self,src_embed,encoder,decoder,tgt_embed,generator):
    super().__init__()

    self.encoder=encoder
    self.decoder=decoder
    self.src_embed=src_embed
    self.tgt_embed=tgt_embed
    self.generator=generator

  def forward(self,src,tgt,src_mask,tgt_mask):
    return EnDcoder(EnDcoder(src,src_mask),src_mask,tgt,tgt_mask)

  def encode(self,src,src_mask):
    return EnDcoder(self.src_embed(src),src_mask)

  def decode(self,memory,src_mask,tgt,tgt_mask):
    return EnDcoder(memory,src_mask,self.tgt_embed(tgt),tgt_mask)

class Generator(nn.Module):
  def __init__(self,d_model,vocab_size):
    super().__init__()
    self.proj=nn.Linear(d_model,vocab_size)

  def forward(self,x):
    return F.log_softmax(self.proj(x),dim=-1)

class EnDcoder(nn.Module):
  def __init__(self,layer,N):
    super().__init__()
    self.layers=clone(layer,N)
    self.norm=nn.LayerNorm(layer.d_model)

  def forward(self,x,src_mask,tgt=None,tgt_mask=None):
    # Encoder model
    if tgt==tgt_mask==None:
      for layer in self.layers:
        x=layer(x,src_mask)
      return self.norm(x)

    # Decoder model
    elif tgt is not None and tgt_mask is not None:
      for layer in self.layers:
        x=layer(x,src_mask,tgt,tgt_mask)
      return self.norm(x)

    # Wrong input
    else:
      raise ValueError("Encoder instance input error: "\
               "tgt and tgt_mask should be both None or both not None.")

def clone(layer,N):
  return nn.ModuleList([copy.deepcopy(layer) for _ in range(N)])

class EnDcoderLayer(nn.module):
  def __init__(self,)

In [5]:
for i in range(2):
  print("a"\
      "b")

ab
ab
